In [1]:
# Available features
features = ['market_cap', 'pe_ratio', 'ps_ratio', 'price_to_book_value',
            'ev_ebitda', 'ev_ebit', 'ev_revenues', 'ev_free_cash_flow',
            'eps', 'ebitda', 'revenues_growth', 'profit_margin', 'cash_on_hand',
            'price_to_cash_flow_ttm', 'peg_ratio', 'ev_assets']

In [2]:
def GetHistoricalFundamentalData(stock, start, end, features):
    import requests
    import time
    import re
    import datetime
    import pandas as pd
    from collections import defaultdict
    
    dic = defaultdict()
    for parameter in features[:]:
        print("----------Working on %s----------" %parameter)
        Charts = 'https://ycharts.com/charts/fund_data.json?securities=include%3Atrue%2Cid%3A{stock}%2C%2C&calcs=include%3Atrue%2Cid%3A{parameter}%2C%2C&correlations=&format=real&recessions=false&zoom=custom&startDate={start}&endDate={end}&chartView=&splitType=single&scaleType=linear&note=&title=&source=false&units=false&quoteLegend=true&partner=&quotes=&legendOnChart=true&securitylistSecurityId=&clientGroupLogoUrl=&displayTicker=false&ychartsLogo=&useEstimates=false&maxPoints=918'.format(stock=stock,parameter=parameter,start=start,end=end)
        headers = {
                'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36',
                ######## You have to update cookies if it doesn't work #########
                'cookie': 'hblid=w4r6qCrL7PNMMxQm1y8Lx0XoAB63J0PA; hubspotutk=461fce12833a01321272e68bb33abdc8; _cb_ls=1; _cb=DM_jLbDae7DuDBPUdz; olfsk=olfsk6171775466330669; 33e807c05af9078f6b2ed01ced5fc28d5c8f52f4=1; __utma=69688216.236940170.1551736858.1551779122.1551808677.6; __utmc=69688216; __utmz=69688216.1551808677.6.3.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); wcsid=vnmOzIbhUrriMHxF1y8Lx0XA63rAaEzo; _okdetect=%7B%22token%22%3A%2215518086775140%22%2C%22proto%22%3A%22https%3A%22%2C%22host%22%3A%22ycharts.com%22%7D; __hstc=165832289.461fce12833a01321272e68bb33abdc8.1551736858200.1551779122558.1551808677535.6; __hssrc=1; _cb_svref=https%3A%2F%2Fwww.google.com%2F; _ok=1228-592-10-8601; _okbk=cd5%3Davailable%2Ccd4%3Dtrue%2Cvi5%3D0%2Cvi4%3D1551808677753%2Cvi3%3Dactive%2Cvi2%3Dfalse%2Cvi1%3Dfalse%2Ccd8%3Dchat%2Ccd6%3D0%2Ccd3%3Dfalse%2Ccd2%3D0%2Ccd1%3D0%2C; page_view_ctr=21; _oklv=1551808686892%2CvnmOzIbhUrriMHxF1y8Lx0XA63rAaEzo; ycsessionid=idkbxh1odgzmi9hqi9i9fjuifoiztlyx; csrftoken=N8XozuwOSfsgYoifRug23LpFDC0npHPw; __utmt=1; __utmb=69688216.10.10.1551808677; mp_bd6455515e9730c7dc2f008755a4ddfe_mixpanel=%7B%22distinct_id%22%3A%20%221694ab962af17-0b23b69d91b4ea-36647105-1aeaa0-1694ab962b14e9%22%2C%22%24device_id%22%3A%20%221694ab962af17-0b23b69d91b4ea-36647105-1aeaa0-1694ab962b14e9%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__mps%22%3A%20%7B%7D%2C%22__mpso%22%3A%20%7B%7D%2C%22__mpus%22%3A%20%7B%7D%2C%22__mpa%22%3A%20%7B%7D%2C%22__mpu%22%3A%20%7B%7D%2C%22__mpr%22%3A%20%5B%5D%2C%22__mpap%22%3A%20%5B%5D%2C%22%24user_id%22%3A%20%221694ab962af17-0b23b69d91b4ea-36647105-1aeaa0-1694ab962b14e9%22%7D; __hssc=165832289.9.1551808677535; _chartbeat2=.1551736858253.1551810188863.11.C4QjlmJB5_0CJswR3Cyf2GcBaTbEs.9; _chartbeat5=',
                'Host': 'ycharts.com',
                'startDate': start,     
                'endDate': end,
                'zoom': 'custom',
                'Pragma': 'no-cache',
                'referer': 'https://ycharts.com/companies/{stock}/pe_ratio'.format(stock=stock),
                'X-Requested-With': 'XMLHttpRequest'
            }

        data = {'securities': 'include:true,id:{stock},,'.format(stock=stock), 
                'calcs': 'include:true,id:{parameter},,'.format(parameter=parameter),
                'maxPoints': '10000'}

        response = requests.get(Charts, headers=headers, data=data)

        text = response.text

        ret = re.match('.+(\[\[.+?\]\])', text)
        ret_ = ret.group(1).split('], ')
        result = [ret_[0][2:]] + [x[1:] for x in ret_[1:-1]] + [ret_[-1][1:-2]]
        date = [datetime.datetime.strptime(time.strftime("%d %m %Y", time.gmtime(int(x) / 1000.0)), '%d %m %Y') for x in [x.split(', ')[0] for x in result]]
        data = [x.split(', ')[1] for x in result]
        
        print('-----Number of Data:', len(date))
        
        for x in date:
            if x not in dic.keys():
                dic[x] = defaultdict()
        
        for x,y in zip(date,data):
            dic[x][parameter] = y
            
        time.sleep(1)
        
    df = pd.DataFrame.from_dict(dic, orient='index')
    
    return df


In [3]:
stock = 'NFLX'
start = '03/03/2010'
end   = '03/03/2015'
features = ['market_cap', 'pe_ratio', 'ps_ratio', 'price_to_book_value',
            'ev_ebitda', 'ev_ebit', 'ev_revenues', 'ev_free_cash_flow']
df = GetHistoricalFundamentalData(stock, start, end,features)

----------Working on market_cap----------
-----Number of Data: 262
----------Working on pe_ratio----------
-----Number of Data: 262
----------Working on ps_ratio----------
-----Number of Data: 262
----------Working on price_to_book_value----------
-----Number of Data: 262
----------Working on ev_ebitda----------
-----Number of Data: 262
----------Working on ev_ebit----------
-----Number of Data: 262
----------Working on ev_revenues----------
-----Number of Data: 262
----------Working on ev_free_cash_flow----------
-----Number of Data: 262


In [4]:
df.head()

,market_cap,pe_ratio,ps_ratio,price_to_book_value,ev_ebitda,ev_ebit,ev_revenues,ev_free_cash_flow
2010-03-03,3616.1725,34.1162,2.3628,18.1587,9.0638,17.5969,2.0930,12.5333
2010-03-10,3815.8515,36.0000,2.4932,19.1614,9.5815,18.6020,2.2126,13.2492
2010-03-17,3796.5798,35.8182,2.4806,19.0646,9.5316,18.5050,2.2010,13.1801
2010-03-24,3775.2894,36.5505,2.5314,18.9577,9.4764,18.3979,2.1883,13.1038
2010-03-31,3853.7894,33.5182,2.3724,26.2636,8.9833,17.2233,2.1400,13.0206


In [ ]:
stock = 'NFLX'
start = '03/03/2010'
end   = '03/03/2015'
features = ['eps', 'ebitda', 'revenues_growth', 'profit_margin','ev_assets']
df = GetHistoricalFundamentalData(stock, start, end,features)

----------Working on eps----------


In [ ]:
df.head()